In [7]:
import geemap
import ee

In [8]:
Map = geemap.Map()
aoi = ee.Geometry.Polygon(
         [[[-51.26670684036638, 71.10667904834996],
           [-50.613642172036386, 69.35172505065574],
           [-51.925886527866396, 67.4097894479071],
           [-51.20992413915373, 66.27518798747208],
           [-51.86713118464301, 66.04083709002118],
           [-51.68516540592766, 65.85920268639616],
           [-50.14610137161638, 63.77010325395928],
           [-44.45518340286638, 63.62039597689339],
           [-44.45518340286638, 71.06394313461017]]]) # dark zone connected ice

# Display AOI on the map.
Map.centerObject(aoi, 4)
# Map.setOptions('HYBRID')
# Map.addLayer(aoi, {'color': 'f8766d'}, 'AOI')



In [11]:
date_start = ee.Date.fromYMD(2009, 8, 16)
date_end = ee.Date.fromYMD(2017, 3, 12)

# greenlandmask = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK') \
#                       .select('ice_mask').eq(1)


In [14]:
elevation = ee.ImageCollection('UMN/PGC/ArcticDEM/V3/2m') \
                  .filterBounds(aoi) \
                  .select('elevation')
elevationVis = {
  'min': -50.0,
  'max': 1000.0,
  'palette': ['0d13d8', '60e1ff', 'ffffff'],
}

# Difference in days between start and finish
diff = date_end.difference(date_start, 'day')

# Make a list of all dates
dayNum = 1; # steps of day number

def range_func(day):
  return date_start.advance(day,'day')

range = ee.List.sequence(0, diff.subtract(1), dayNum).map(range_func)

# Function for iteration over the range of dates
def day_mosaicsArcticDEM(date, newlist):
  # Cast
  date = ee.Date(date)
  newlist = ee.List(newlist)

  # Filter collection between date and the next day
  filtered = elevation.filterDate(date, date.advance(dayNum,'day'))
  # Make the mosaic
  image = ee.Image(
      filtered.mosaic().copyProperties(filtered.first())) \
      .set({'date': date.format('yyyy-MM-dd')}) \
      .set('system:time_start', filtered.first().get('system:time_start'))
  
  # Add the mosaic to a list only if the collection has images
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))

arcticDEMdaily = ee.ImageCollection(ee.List(range.iterate(day_mosaicsArcticDEM, ee.List([]))))

Map.addLayer(arcticDEMdaily, {}, "elevation", False)
Map.add_time_slider(arcticDEMdaily, elevationVis)
Map

Map(bottom=1003.0, center=[67.23806155909902, -47.81250000000001], controls=(WidgetControl(options=['position'…